In [ ]:
%config Completer.use_jedi = False

import torch
import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw

# # Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path = 'best-new.pt')



model.half()
model.conf = 0.75
model.iou = 0.5

In [ ]:
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot
import traitlets
import ipywidgets
import cv2
import numpy as np
import pandas
import time
from jetbot import Robot

In [ ]:
# Displaying camera output

camera = Camera.instance(width=300, height=300, fps= 3)


image_widget = ipywidgets.Image()  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

In [ ]:
robot = Robot()

In [ ]:
obstacle_detected = False
motor_values = []
forward_counter = 0
forward_delay = 10


def detect(img, model):
    global obstacle_detected, motor_values, forward_counter
    classes = ['obstacle']
    # with open('coco.names', 'r') as f:
    #     classes = [line.strip() for line in f.readlines()]

    # Load an image for detection
#     img = Image.open('img.jpg')
    img = Image.fromarray(img)
    # Transform the image
    transform = transforms.Compose([
        transforms.Resize((416, 416)),
        transforms.ToTensor(),
    #     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img_tensor = transform(img)

    #Perform object detection on the image
    results = model(img)

    # Extract the detected objects and their bounding boxes, class IDs, and confidence scores
    boxes = results.xyxy[0].cpu().numpy()[:, :4]
    class_ids = results.pred[0].cpu().numpy()[:, 5].astype(int)
    confidences = results.pred[0].cpu().numpy()[:, 4]

    # Apply non-max suppression to remove overlapping bounding boxes
    indices = torchvision.ops.nms(torch.from_numpy(boxes), torch.from_numpy(confidences), iou_threshold=0.5)

    # Draw bounding boxes on the image for objects with confidence score above the threshold
    colors = torch.tensor([[0, 255, 0], [0, 0, 255], [255, 0, 0]])
    print(motor_values)
    if len(indices) > 0:
        obstacle_detected = True
        forward_counter = 0
        
        draw = ImageDraw.Draw(img)
        for i in indices:
            x1, y1, x2, y2 = boxes[i]
            label = f'{classes[class_ids[i]]} {confidences[i]:.2f}'
            color = colors[class_ids[i]]
            draw.rectangle([x1, y1, x2, y2], width=2, outline=tuple(color))
            draw.text((x1, y1 - 10), label, fill=tuple(color))
            
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            draw.ellipse([center_x-2, center_y-2, center_x+2, center_y+2], fill=tuple(color), outline=tuple(color))
            
            im_width, im_height = img.size
            draw.ellipse([im_width//2-2, im_height//2-2, im_width//2+2, im_height//2+2], fill=(255, 255, 255), outline=(255, 255, 255))
            error = (center_x - im_width/2) / (im_width/2)
            height = y2 - y1
#             print("height : " + str(height))
#             print("Error : "+str(error))
            
            # Initiate the turn when the bounding box height reaches some threshold.
            if height <=70:
                robot.forward(0.4)
            else:
                robot.stop()
                motor_value = 0.4 +  0.4 * (1 + error)
#                 motor_value= max(0, min(1, motor_value))
                robot.left_motor.value = motor_value
                motor_values.append((robot.left_motor.value, robot.right_motor.value))
    elif len(motor_values) > 0:
        forward_counter += 1
        if forward_counter <= forward_delay:
            robot.left_motor.value = 0.4
            robot.right_motor.value = 0.4
        else: 
            print("Inside else block")
            print(motor_values)
            left_motor, right_motor = motor_values.pop(0)
            print("After pop")
            print(motor_values)
            print("After motor values")
            
            robot.left_motor.value = right_motor
            robot.right_motor.value = left_motor
            time.sleep(0.3)
            
            if len(motor_values) == 0:
                robot.stop()
    else:
        robot.stop()
                
                

            # Add a flag here to initate the turning manuver. 
            # After the obejct is out of the frmae, reverse the order of the commands and get it to the line. 
            # add line following to this. 
    # Display the image with bounding boxes

#     if obstacle_detected == True and len(indices) == 0:
#         robot.forward(0.4)
#         time.sleep(2.5)
#         robot.stop()
#         robot.right_motor.value = 0.5
#         time.sleep(3)
#         obstacle_detected = False
        
        
        
    return np.array(img)

In [ ]:
## test_mode = btntoggle.value
# test_mode = False # If test mode is enabled robot will not move 
robot_mode = "INITIALIZE"
# robot_mode = "GO" # WAIT, GO,
base_motor_speed = 0.5 # speed varies beteen 0 to 1.
# line pos 1 = 280, pos 2 = 200, pos 3 = 295
line_pos_1 = 270
line_pos_2 = 230
line_pos_3 = 230
verticle_line_pos_1 = 75
verticle_line_pos_2 = 225
line_color_set = 255
mid_point = 150
Kp = 0.8
Kd = 0.1
Ki = 0
frame_counter = 0
stop_counter = 0
storage_id = 4

robot = Robot()

previous_error = 0
I = 0

obstacle_detected = False
motor_values = []
forward_counter = 0
forward_delay = 10
flush_counter = 0
flush_delay = 20

hue_min = 10
hue_max = 100

saturation_min = 10
saturation_max = 100

value_min = 10
value_max = 100

consecutive_obstacle_frames = 0

def check_obstacle_detected():
    global consecutive_obstacle_frames
    if consecutive_obstacle_frames >= 10:
        return True
    else:
        return False


def detect(img, model):
    global obstacle_detected, motor_values, forward_counter
    classes = ['obstacle']
    # with open('coco.names', 'r') as f:
    #     classes = [line.strip() for line in f.readlines()]

    # Load an image for detection
#     img = Image.open('img.jpg')
    img = Image.fromarray(img)
    # Crop the top portion of the image
#     cropped_img = img.crop((0, crop_height, img.width, img.height))

#     # Create a black image with the same size as the cropped region
#     black_pixels = np.zeros_like(cropped_img)

#     # Concatenate the black pixels and the remaining image
#     new_img = Image.fromarray(np.concatenate((black_pixels, cropped_img), axis=0))

    # Transform the image
    transform = transforms.Compose([
        transforms.ToTensor(),
    #     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img_tensor = transform(img)

    #Perform object detection on the image
    results = model(img)

    # Extract the detected objects and their bounding boxes, class IDs, and confidence scores
    boxes = results.xyxy[0].cpu().numpy()[:, :4]
    class_ids = results.pred[0].cpu().numpy()[:, 5].astype(int)
    confidences = results.pred[0].cpu().numpy()[:, 4]

    # Apply non-max suppression to remove overlapping bounding boxes
    indices = torchvision.ops.nms(torch.from_numpy(boxes), torch.from_numpy(confidences), iou_threshold=0.5)

    # Draw bounding boxes on the image for objects with confidence score above the threshold
    colors = torch.tensor([[0, 255, 0], [0, 0, 255], [255, 0, 0]])
    print(motor_values)
    if len(indices) > 0:
        print("obs detected")
        obstacle_detected = True
        
        
        forward_counter = 0
        
        draw = ImageDraw.Draw(img)
        for i in indices:
            x1, y1, x2, y2 = boxes[i]
            label = f'{classes[class_ids[i]]} {confidences[i]:.2f}'
            color = colors[class_ids[i]]
            draw.rectangle([x1, y1, x2, y2], width=2, outline=tuple(color))
            draw.text((x1, y1 - 10), label, fill=tuple(color))
            
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            draw.ellipse([center_x-2, center_y-2, center_x+2, center_y+2], fill=tuple(color), outline=tuple(color))
            
            im_width, im_height = img.size
            draw.ellipse([im_width//2-2, im_height//2-2, im_width//2+2, im_height//2+2], fill=(255, 255, 255), outline=(255, 255, 255))
            error = (center_x - im_width/2) / (im_width/2)
            height = y2 - y1
#             print("height : " + str(height))
#             print("Error : "+str(error))
            
            # Initiate the turn when the bounding box height reaches some threshold.
#             if height <=70:
#                 robot.forward(0.4)
#             else:
#                 robot.stop()
#                 motor_value = 0.4 +  0.4 * (1 + error)
# #                 motor_value= max(0, min(1, motor_value))
#                 robot.left_motor.value = motor_value
#                 motor_values.append((robot.left_motor.value, robot.right_motor.value))
    elif len(motor_values) > 0:
        print("inside len(motor_values)")
#         forward_counter += 1
#         if forward_counter <= forward_delay:
#             robot.left_motor.value = 0.4
#             robot.right_motor.value = 0.4
#         else: 
#             print("Inside else block")
#             print(motor_values)
#             left_motor, right_motor = motor_values.pop(0)
#             print("After pop")
#             print(motor_values)
#             print("After motor values")
            
#             robot.left_motor.value = right_motor
#             robot.right_motor.value = left_motor
#             time.sleep(0.3)
            
#             if len(motor_values) == 0:
#                 robot.stop()
    else:
        print("inside else loop")
#         robot.stop()
        obstacle_detected = False
                
                

            # Add a flag here to initate the turning manuver. 
            # After the obejct is out of the frmae, reverse the order of the commands and get it to the line. 
            # add line following to this. 
    # Display the image with bounding boxes

#     if obstacle_detected == True and len(indices) == 0:
#         robot.forward(0.4)
#         time.sleep(2.5)
#         robot.stop()
#         robot.right_motor.value = 0.5
#         time.sleep(3)
#         obstacle_detected = False
        
        
        
    return np.array(img), indices




def follow_line(imageInput, test_mode, dual_motor, lower_beige, upper_beige):
#     print(robot_mode)
    global previous_error, I, robot_mode, frame_counter, stop_counter, storage_id, obstacle_detected, consecutive_obstacle_frames, motor_values, forward_counter, flush_counter
    
    
    
    cam_image = camera.value
    img = cv2.resize(cam_image, (300, 300))

    # Convert the image to the desired color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

#     lower_beige = np.array([0, 10, 130])
#     upper_beige = np.array([200, 255, 250])
    
    
    # Create a mask based on the specified color range
    mask = cv2.inRange(hsv, lower_beige, upper_beige)

    # Apply the mask to the original image
    result = cv2.bitwise_and(img, img, mask=mask)

    # Display the original image and the result
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    
    eroded = cv2.erode(gray, None, iterations = 5)
    dilated = cv2.dilate(eroded, None, iterations = 6)
    _, thresh = cv2.threshold(dilated, 100, 255, cv2.THRESH_OTSU)

#     _, thresh = cv2.threshold(dilated, 0, 255, cv2.THRESH_BINARY)

    
#     img = camera.value
#     img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     gray_de_noised = cv2.GaussianBlur(gray, (3, 3), 0)


#     eroded = cv2.erode(gray_de_noised, None, iterations = 8)
#     dilated = cv2.dilate(eroded, None, iterations = 5)
# #     ret, thresh = cv2.threshold(dilated, 100, 255, cv2.THRESH_OTSU)
#     ret, thresh = cv2.threshold(dilated, 120, 255, cv2.THRESH_BINARY)
    
    ##### Detection ###########################################################################################################################
    
    classes = ['obstacle']
    
    img = Image.fromarray(img)
    img_array = np.array(img)

    # Replace the top 10 pixel lines with black pixel values
    img_array[:30, :] = 0

    # Convert the modified array back to an image
    modified_img = Image.fromarray(img_array)

#     cropped_img = img.crop((0, 0, img.width, crop_height))
#     cropped_img = img.crop((0, 0, img.width, 100))

#     # Create a black image with the same size as the cropped region
#     black_pixels = np.zeros_like(cropped_img)

    # Concatenate the black pixels and the remaining image
#     new_img = Image.fromarray(np.concatenate((black_pixels, cropped_img), axis=0))
    new_img =  Image.fromarray(img_array)



    # Transform the image
    transform = transforms.Compose([
        transforms.ToTensor(),
    #     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img_tensor = transform(new_img)

    #Perform object detection on the image
    results = model(new_img)

    # Extract the detected objects and their bounding boxes, class IDs, and confidence scores
    boxes = results.xyxy[0].cpu().numpy()[:, :4]
    class_ids = results.pred[0].cpu().numpy()[:, 5].astype(int)
    confidences = results.pred[0].cpu().numpy()[:, 4]

    # Apply non-max suppression to remove overlapping bounding boxes
    indices = torchvision.ops.nms(torch.from_numpy(boxes), torch.from_numpy(confidences), iou_threshold=0.5)
    
    # Draw bounding boxes on the image for objects with confidence score above the threshold
    colors = torch.tensor([[0, 255, 0], [0, 0, 255], [255, 0, 0]])
    if len(indices) > 0:
        print("obs detected")
        obstacle_detected = True
        
        draw = ImageDraw.Draw(new_img)
        for i in indices:
            x1, y1, x2, y2 = boxes[i]
            label = f'{classes[class_ids[i]]} {confidences[i]:.2f}'
            color = colors[class_ids[i]]
            draw.rectangle([x1, y1, x2, y2], width=2, outline=tuple(color))
            draw.text((x1, y1 - 10), label, fill=tuple(color))
            
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            draw.ellipse([center_x-2, center_y-2, center_x+2, center_y+2], fill=tuple(color), outline=tuple(color))
            
            im_width, im_height = new_img.size
            draw.ellipse([im_width//2-2, im_height//2-2, im_width//2+2, im_height//2+2], fill=(255, 255, 255), outline=(255, 255, 255))
    else:
        obstacle_detected = False
    
    det_output = np.array(new_img)
    
    if obstacle_detected:
        consecutive_obstacle_frames += 1
#     else:
#         consecutive_obstacle_frames = 0
        
    print("consec obs: " + str(consecutive_obstacle_frames))
    obstacle_flag = check_obstacle_detected()
    print(obstacle_flag)
#     if obstacle_detected == False:
    if not obstacle_flag:
    
#         img = camera.value
#         img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         gray_de_noised = cv2.GaussianBlur(gray, (3, 3), 0)


#         eroded = cv2.erode(gray_de_noised, None, iterations = 8)
#         dilated = cv2.dilate(eroded, None, iterations = 5)
#         ret, thresh = cv2.threshold(dilated, 0, 255, cv2.THRESH_OTSU)
    #     ret, thresh = cv2.threshold(dilated, 100, 255, cv2.THRESH_BINARY)



        color_pos_1 = thresh[line_pos_1]
        color_pos_2 = thresh[line_pos_2]
        color_pos_3 = thresh[line_pos_3]

        trimmed_line_1 = color_pos_1[verticle_line_pos_1:verticle_line_pos_2]
        trimmed_line_2 = color_pos_2[verticle_line_pos_1:verticle_line_pos_2]
        trimmed_line_3 = color_pos_3[verticle_line_pos_1:verticle_line_pos_2]
        try:
            line_color_count_pos_1 = np.sum(trimmed_line_1 == line_color_set)
            line_color_count_pos_2 = np.sum(trimmed_line_2 == line_color_set)

            line_index_pos_1 = np.where(trimmed_line_1 == line_color_set)
            line_index_pos_2 = np.where(trimmed_line_2 == line_color_set)

            if line_color_count_pos_1 == 0:
                line_color_count_pos_1 = 1
            if line_color_count_pos_2 == 0:
                line_color_count_pos_2 = 1

            right_pos_1 = line_index_pos_1[0][-1] + verticle_line_pos_1
            left_pos_1 = line_index_pos_1[0][0] + verticle_line_pos_1
            center_pos_1 = int((left_pos_1 + right_pos_1)/2)

            right_pos_2 = line_index_pos_2[0][-1] + verticle_line_pos_1
            left_pos_2 = line_index_pos_2[0][0] + verticle_line_pos_1
            center_pos_2 = int((left_pos_2 + right_pos_2)/2)


            center = int((center_pos_1 + center_pos_2)/2)
        except Exception as err:
            center = None
            pass
        thresh = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)
        text = str(center)
        coordinates = (100,100)
        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 1
        color = (0, 0, 255)
        thickness = 2


        try:
            # Box for region of interest
            cv2.line(thresh, (verticle_line_pos_1, line_pos_1), (verticle_line_pos_2, line_pos_1), (255,255,64),1)
            cv2.line(thresh, (verticle_line_pos_1, line_pos_2), (verticle_line_pos_2, line_pos_2), (255,255,64),1)

            cv2.line(thresh, (verticle_line_pos_1, line_pos_1), (verticle_line_pos_1, line_pos_2), (255,255,64),1)
            cv2.line(thresh, (verticle_line_pos_2, line_pos_1), (verticle_line_pos_2, line_pos_2), (255,255,64),1)

            # Middle line
            cv2.line(thresh, (150, 0), (150, 640), (0,0,255),1)

            # Lines for intersections of the box
            cv2.line(thresh, (left_pos_1, (line_pos_1 + 10)), (left_pos_1, (line_pos_1 - 10)), (255,128,64),1)
            cv2.line(thresh, (right_pos_1, (line_pos_1 + 10)), (right_pos_1, (line_pos_1 - 10)), (64,128,255),1)
            cv2.line(thresh, (left_pos_2, (line_pos_2 + 10)), (left_pos_2, (line_pos_2 - 10)), (255,128,64),1)
            cv2.line(thresh, (right_pos_2, (line_pos_2 + 10)), (right_pos_2, (line_pos_2 - 10)), (64,128,255),1)

            # Center marker
            cv2.line(thresh, ((center - 20), int((line_pos_1 + line_pos_2)/2)), ((center + 20), int((line_pos_1 + line_pos_2)/2)), (0,0,255), 1)
            cv2.line(thresh, ((center), int((line_pos_1 + line_pos_2)/2 + 20)), ((center), int((line_pos_1 + line_pos_2)/2 - 20)), (0,0,255), 1)

        except:
            pass

        if center != None:

            direction = ""
            if 145 < center < 155 :
                direction = "Go straight"
            elif center < 145 :
                direction = "Turn left"
            elif 155 < center :
                direction = "Turn right"

            trimmed_line_mid_point = int(len(trimmed_line_3)/2)
    #         left_turn = np.all(trimmed_line_3[:trimmed_line_mid_point] == 0) and np.all(trimmed_line_3[-30:-10] == 255)
    #         right_turn = np.all(trimmed_line_3[trimmed_line_mid_point:] == 0) and np.all(trimmed_line_3[10:30] == 255)
            left_turn = np.all(trimmed_line_3[:trimmed_line_mid_point] == 0) 
            right_turn = np.all(trimmed_line_3[trimmed_line_mid_point:] == 0)
            if True:
            
                error = (center - mid_point)/mid_point
                P = error
                I = I + error
                D = error - previous_error
                PID_value = Kp*P + Kd*D + Ki*I
                previous_error = error
    #             thresh = cv2.putText(thresh, str(PID_value), (150,40), font, fontScale, color, thickness, cv2.LINE_AA)



                if not test_mode:
                    if robot_mode == "INITIALIZE":
                        global x,y

                        robot_mode = "GO"
                    elif robot_mode == "GO":
                        if dual_motor:
                            robot.left_motor.value = base_motor_speed + PID_value
                            robot.right_motor.value = base_motor_speed - PID_value
                        else:
                            if PID_value > 0:
                                robot.left_motor.value = base_motor_speed + PID_value
                                robot.right_motor.value = base_motor_speed 
                            elif PID_value <= 0 :
                                robot.left_motor.value = base_motor_speed 
                                robot.right_motor.value = base_motor_speed - PID_value

                else:
                    robot.stop()  

            thresh = cv2.putText(thresh, robot_mode, (10,30), font, fontScale, color, thickness, cv2.LINE_AA)
        else:
            robot.stop()
    else:
        if len(indices) > 0:
            print("obs detected")
            obstacle_detected = True


            forward_counter = 0
            flush_counter = 0

            draw = ImageDraw.Draw(img)
            for i in indices:
                x1, y1, x2, y2 = boxes[i]
                label = f'{classes[class_ids[i]]} {confidences[i]:.2f}'
                color = colors[class_ids[i]]
                draw.rectangle([x1, y1, x2, y2], width=2, outline=tuple(color))
                draw.text((x1, y1 - 10), label, fill=tuple(color))

                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                draw.ellipse([center_x-2, center_y-2, center_x+2, center_y+2], fill=tuple(color), outline=tuple(color))

                im_width, im_height = img.size
                draw.ellipse([im_width//2-2, im_height//2-2, im_width//2+2, im_height//2+2], fill=(255, 255, 255), outline=(255, 255, 255))
                error = (center_x - im_width/2) / (im_width/2)
                height = y2 - y1
    #             print("height : " + str(height))
    #             print("Error : "+str(error))

                # Initiate the turn when the bounding box height reaches some threshold.
                if height <=70:
                    robot.forward(0.4)
                else:
                    robot.stop()
                    motor_value = 0.4 +  0.4 * (1 + error)
    #                 motor_value= max(0, min(1, motor_value))
                    robot.left_motor.value = motor_value
                    motor_values.append((robot.left_motor.value, robot.right_motor.value))
        elif len(motor_values) > 0:
            print("inside len(motor_values)")
            forward_counter += 1
            if forward_counter <= forward_delay:
                robot.left_motor.value = 0.4
                robot.right_motor.value = 0.4
#                 motor_values.append((robot.left_motor.value, robot.right_motor.value))

            else: 
                print("Inside else block")
                print(motor_values)
                left_motor, right_motor = motor_values.pop(0)
                print("After pop")
                print(motor_values)
                print("After motor values")
                robot.left_motor.value = right_motor
                robot.right_motor.value = left_motor
                time.sleep(0.3)
                
#                 if len(motor_values) > 2:
#                     robot.left_motor.value = right_motor
#                     robot.right_motor.value = left_motor
#                     time.sleep(0.3)
#                 else:
#                     robot.left_motor.value = 0
#                     robot.right_motor.value = 0
#                     obstacle_detected = False
#                     consecutive_obstacle_frames = 0
#                 if len(motor_values) == 0:
#                     robot.stop()
                if len(motor_values) == 0:
                    robot.stop()
        else:
            print("inside else loop")
            flush_counter += 1
            if flush_counter <= flush_delay:
                robot.left_motor.value = 0.4
                robot.right_motor.value = 0.4
            else:
    #         robot.stop()
#             robot.left_motor.value = 0.4
#             robot.right_motor.value = 0.4
#             time.sleep(4)
                obstacle_detected = False
                consecutive_obstacle_frames = 0
                robot.stop()

#     return thresh, det_output
    return thresh, result, det_output